# PM Tiles demo

Denne notebooken er ment å vise frem kapabilitetene i PM Tiles formatet.

## Systemkrav

Demoen krever noen verktøy som ikke kan håndteres isolert i et python miljø,
for at dette skal være så portabelt som mulig er det lagt opp til å bruke Docker
til å håndtere miljøet for disse verktøyene.

Blokken under sjekker at Docker og Docker-compose er installert på vertssystemet.

In [ ]:
import subprocess
from typing import List

def execute_command(cmd: List[str], **kwargs):
    return subprocess.run(
        cmd,
        capture_output=True,
        text=True,
        **kwargs
    )

system_requirements = [
    ["docker-compose", "--version"],
    ["docker", "--version"],
]

for requirement in system_requirements:
    assert execute_command(requirement, check=False).returncode == 0, f"System requirement {requirement} is not satisfied"

## Input data



### Raster


![nedlasting](../../public/img/geonorge-n50-raster-krs.png)

Last ned filen som vist over, pakk ut og flytt .tif filen til `raster/data/32_N50raster_2024.tif`


### Vektor 

![nedlasting](../../public/img/geonorge-n50-vektor-agder.png)

Last ned filen som vist over, pakk ut og flytt .gdb filen til `vector/data/Basisdata_42_Agder_25832_N50Kartdata_FGDB.gdb`

Først sjekker vi at input filene finnes som forventet.

In [ ]:
import utils

assert utils.path_exists("raster/data/32_N50raster_2024.tif"), f"Mangler inputfil raster/data/32_N50raster_2024.tif"
assert utils.path_exists("vector/data/Basisdata_42_Agder_25832_N50Kartdata_FGDB.gdb"), f"Mangler inputfil vector/data/Basisdata_42_Agder_25832_N50Kartdata_FGDB.gdb"

## Konverterer raster data

Merk at her brukes python komandolinjeverktøyet `rio-pmtiles` som installeres via requirements.txt filen.

```bash
# Etter at virtuelt python miljø er aktivert.
pip install -r requirements.txt
```

In [ ]:
# Oppretter mappe for output
utils.create_dir("raster/out")


In [ ]:
# Genererer pmtiles
!rio pmtiles raster/data/32_N50raster_2024.tif raster/out/N50_raster_2024.pmtiles --format PNG

## Konverterer vektor data

In [ ]:
# Oppretter mappe for output
utils.create_dir("vector/out")

### Dockerfil

For at demoen skal være relativt portabel skjer selve transformasjonen fra gdb til pmtiles
i en docker container gjennom en rekke transformasjoner ved hjelp av flere verktøy.

Format transformasjon:

gdb => geojson => mbtiles => pmtiles

Verktøy:

- ogr2ogr: Konverterer gdb til geojson (gdal)
- tippecanoe: Konverterer geojson til mbtiles
- pmtiles: Konverterer mbtiles til pmtiles


Det er mulig å kombinere flere lag i samme .pmtiles fil, men ved bruk av `vector/build.sh` scriptet fungerte
ikke dette helt som forventet. Lagene blir opprettet, men ved lavere zoom ser ikke kartet pent ut. Videreutvikling
oppfordres.

In [ ]:
# Bygger docker image med gdal, tippecanoe og pmtiles
!docker build -t pmtiles -f vector/Dockerfile .

Starter en container for å kunne kopiere lokale filer over til containeren og kjøre konverteringsscriptet
i det isolerte miljøet.

In [ ]:
!docker run --name pmtiles-converter -d pmtiles

Kopierer kildedata inn i containeren.

In [ ]:
!docker cp ./vector/data/Basisdata_42_Agder_25832_N50Kartdata_FGDB.gdb pmtiles-converter:/app/N50_vektor_agder.gdb 

Konverterer samferdsel senterlinje laget til pmtiles

In [ ]:
!docker exec pmtiles-converter bash gdb_to_pmtiles.sh N50_vektor_agder.gdb N50_Samferdsel_senterlinje N50_Samferdsel_senterlinje.pmtiles

Kopierer pmtiles filen ut av containeren

In [ ]:
!docker cp pmtiles-converter:/app/N50_Samferdsel_senterlinje.pmtiles ./vector/out/N50_Samferdsel_senterlinje.pmtiles

Konverterer bygninger og anlegg område laget til pmtiles

In [ ]:
!docker exec pmtiles-converter bash gdb_to_pmtiles.sh N50_vektor_agder.gdb N50_BygningerOgAnlegg_omrade N50_BygningerOgAnlegg_omrade.pmtiles

Kopierer pmtiles filen ut av containeren

In [ ]:
!docker cp pmtiles-converter:/app/N50_BygningerOgAnlegg_omrade.pmtiles ./vector/out/N50_BygningerOgAnlegg_omrade.pmtiles

Stopper og fjerner containeren etter at konverteringen er ferdig.

In [ ]:
!docker stop pmtiles-converter
!docker rm pmtiles-converter

Fjerner docker image

In [ ]:
!docker rmi pmtiles

# Demo

I denne delen bruker vi `docker-compose` til å kjøre en lokal demo hvor pmtiles filene hostes
over nettet og en veldig enkel web-app leser data fra filene.

Demoen består av to applikasjoner:

- Web appen som kjører i nettleseren.
- En ngix container som hoster de tre pmtiles filene vi har generert.

Mens docker compose blokken under kjører kan web appen åpnes på 
[http://localhost:3000](http://localhost:3000).
I appen vil man se de tre filene vi har generert som lag, det kreves ingen back-end prosessering utover
å tilgjengeliggjøre filene over internett. Kildekoden for web appen finnes under `client/` mappen.
Ved å inspisere nettverkstrafikken i nettleseren kan man observere 
hvordan range headeren brukes til å spørre ut deler av filene.

(Demo blokken må stoppes manuelt)

In [ ]:
!docker compose up --build

Fjerner demo containere og images

In [ ]:
!docker compose rm -f -s -v